**Data Processing for Car-Dheko Used Car Price Prediction**

In [28]:
import pandas as pd

file_path = r"C:\Users\USER\Desktop\Car-Dheko_Used_Car_Price_Prediction\csv_files\bangalore_cars.xlsx"

# Get all sheet names
sheets = pd.ExcelFile(file_path).sheet_names
print("Available sheets:", sheets)


Available sheets: ['bangalore_cars.csv']


In [31]:
df1 = pd.read_excel(file_path, sheet_name="bangalore_cars.csv")  # or sheet_name=0 for the first sheet
print(df1.head())

                                      new_car_detail  \
0  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
1  {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...   
2  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
3  {'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...   
4  {'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...   

                                    new_car_overview  \
0  {'heading': 'Car overview', 'top': [{'key': 'R...   
1  {'heading': 'Car overview', 'top': [{'key': 'R...   
2  {'heading': 'Car overview', 'top': [{'key': 'R...   
3  {'heading': 'Car overview', 'top': [{'key': 'R...   
4  {'heading': 'Car overview', 'top': [{'key': 'R...   

                                     new_car_feature  \
0  {'heading': 'Features', 'top': [{'value': 'Pow...   
1  {'heading': 'Features', 'top': [{'value': 'Pow...   
2  {'heading': 'Features', 'top': [{'value': 'Pow...   
3  {'heading': 'Features', 'top': [{'value': 'Pow...   
4  {'heading': 'Features', 'top': [{'value': '

Method 1: Display Full Dictionary (No Truncation)

In [38]:
print(df1.columns.tolist())

['car_links', 'detail_it', 'detail_ft', 'detail_bt', 'detail_km', 'detail_transmission', 'detail_ownerNo', 'detail_owner', 'detail_oem', 'detail_model', 'detail_modelYear', 'detail_centralVariantId', 'detail_variantName', 'detail_price', 'detail_priceActual', 'detail_priceSaving', 'detail_priceFixedText', 'detail_trendingText.imgUrl', 'detail_trendingText.heading', 'detail_trendingText.desc', 'new_car_overview_heading', 'new_car_overview_top', 'new_car_overview_bottomData', 'new_car_feature_heading', 'new_car_feature_top', 'new_car_feature_data', 'new_car_feature_commonIcon', 'new_car_specs_heading', 'new_car_specs_top', 'new_car_specs_data', 'new_car_specs_commonIcon', 'first_feature']


In [56]:
# Set Pandas to show full content of dictionaries
pd.set_option('display.max_colwidth', None)  # Disable column width truncation

# Display the 'new_car_detail' column
print(df1['new_car_overview_bottomData'].apply(lambda x: x))

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
30      None
31      None
32      None
33      None
34      None
35      None
36      None
37      None
38      None
39      None
40      None
41      None
42      None
43      None
44      None
45      None
46      None
47      None
48      None
49      None
50      None
51      None
52      None
53      None
54      None
55      None
56      None
57      None
58      None
59      None
60      None
61      None
62      None
63      None
64      None
65      None
66      None
67      None
68      None
69      None
70      None
71      None
72      None
73      None
74      None
75      None
76      None

In [54]:
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481 entries, 0 to 1480
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   car_links                    1481 non-null   object
 1   detail_it                    1481 non-null   int64 
 2   detail_ft                    1481 non-null   object
 3   detail_bt                    1481 non-null   object
 4   detail_km                    1481 non-null   object
 5   detail_transmission          1481 non-null   object
 6   detail_ownerNo               1481 non-null   int64 
 7   detail_owner                 1481 non-null   object
 8   detail_oem                   1481 non-null   object
 9   detail_model                 1481 non-null   object
 10  detail_modelYear             1481 non-null   int64 
 11  detail_centralVariantId      1481 non-null   int64 
 12  detail_variantName           1481 non-null   object
 13  detail_price                 1481

Step 1: Convert JSON strings to Python dictionaries
First, ensure the columns contain actual dictionaries (not strings):

In [32]:
import ast

# Convert stringified JSON to dict (if needed)
for col in ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']:
    if isinstance(df1[col].iloc[0], str):
        df1[col] = df1[col].apply(ast.literal_eval)

Step 2: Flatten nested columns
We'll use json_normalize to unpack dictionaries into separate columns:

In [33]:
from pandas import json_normalize

# Flatten the 'new_car_detail' column
detail_cols = json_normalize(df1['new_car_detail'])
detail_cols.columns = [f"detail_{col}" for col in detail_cols.columns]
df1 = pd.concat([df1, detail_cols], axis=1).drop('new_car_detail', axis=1)

B. Flatten other nested columns
Repeat for other columns (adjust field names as needed):

In [34]:
for col in ['new_car_overview', 'new_car_feature', 'new_car_specs']:
    flattened = json_normalize(df1[col])
    flattened.columns = [f"{col}_{subcol}" for subcol in flattened.columns]
    df1 = pd.concat([df1, flattened], axis=1).drop(col, axis=1)

Step 3: View ALL Expanded Data
To see all rows and columns without truncation:

In [35]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 50)  # Adjust column width

print(df1.head())  # Now shows full expanded data

                                           car_links  detail_it detail_ft  detail_bt detail_km detail_transmission  detail_ownerNo detail_owner detail_oem        detail_model  detail_modelYear  detail_centralVariantId        detail_variantName detail_price detail_priceActual detail_priceSaving detail_priceFixedText                         detail_trendingText.imgUrl detail_trendingText.heading             detail_trendingText.desc new_car_overview_heading                               new_car_overview_top new_car_overview_bottomData new_car_feature_heading                                new_car_feature_top                               new_car_feature_data                         new_car_feature_commonIcon new_car_specs_heading                                  new_car_specs_top                                 new_car_specs_data new_car_specs_commonIcon
0  https://www.cardekho.com/used-car-details/used...          0    Petrol  Hatchback  1,20,000              Manual               3    3rd

In [36]:
# Example: Extract first feature
df1['first_feature'] = df1['new_car_feature_top'].apply(lambda x: x[0]['value'] if x else None)

work

In [2]:
import pandas as pd
import ast
from pandas import json_normalize

# Load the Excel file
file_path = "csv_files/bangalore_cars.xlsx"
df = pd.read_excel(file_path, sheet_name="bangalore_cars.csv")

# Function to convert string to dictionary
def parse_dict_column(column):
    return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Parse each dictionary column
df["new_car_detail"] = parse_dict_column(df["new_car_detail"])
df["new_car_overview"] = parse_dict_column(df["new_car_overview"])
df["new_car_feature"] = parse_dict_column(df["new_car_feature"])
df["new_car_specs"] = parse_dict_column(df["new_car_specs"])

# Normalize and flatten each dictionary
df_banglore_detail = json_normalize(df["new_car_detail"])
df_banglore_overview = json_normalize(df["new_car_overview"].apply(lambda x: {item['key']: item['value'] for item in x.get('top', [])}))
df_banglore_feature = json_normalize(df["new_car_feature"].apply(lambda x: {'Feature_' + str(i): f['value'] for i, f in enumerate(x.get('top', []))}))
df_banglore_specs = json_normalize(df["new_car_specs"].apply(lambda x: {item['key']: item['value'] for item in x.get('top', [])}))

# Add city name
for d in [df_banglore_detail, df_banglore_overview, df_banglore_feature, df_banglore_specs]:
    d["City"] = "bangalore"

# Combine all structured data
df_final = pd.concat([df_banglore_detail, df_banglore_overview, df_banglore_feature, df_banglore_specs], axis=1)

# Save to Excel
df_final.to_excel("structured_bangalore_cars.xlsx", index=False)

print("Data has been structured and saved to 'structured_bangalore_cars.xlsx'")


Data has been structured and saved to 'structured_bangalore_cars.xlsx'


work2

In [7]:
import pandas as pd
import ast
from pandas import json_normalize

# Load the Excel file
file_path = "csv_files/bangalore_cars.xlsx"
df = pd.read_excel(file_path, sheet_name="bangalore_cars.csv")

# Function to convert string to dictionary
def parse_dict_column(column):
    return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Parse dictionary-like columns
df["new_car_detail"] = parse_dict_column(df["new_car_detail"])
df["new_car_overview"] = parse_dict_column(df["new_car_overview"])
df["new_car_feature"] = parse_dict_column(df["new_car_feature"])
df["new_car_specs"] = parse_dict_column(df["new_car_specs"])

# --- Detail flattening ---
df_banglore_detail = json_normalize(df["new_car_detail"])

# --- Overview flattening ---
def flatten_overview(overview_data):
    return {item['key']: item['value'] for item in overview_data.get("top", []) if isinstance(item, dict)}

df_banglore_overview = pd.json_normalize(df["new_car_overview"].apply(flatten_overview))

# --- Feature flattening (flatten all nested lists) ---
def flatten_feature_list(feature_data):
    all_features = []
    if isinstance(feature_data, dict):
        for section in feature_data.values():  # loop over 'top', 'others', etc.
            if isinstance(section, list):
                all_features.extend(section)
    return {'Feature_' + str(i): f.get('value') for i, f in enumerate(all_features)}

df_banglore_feature = pd.json_normalize(df["new_car_feature"].apply(flatten_feature_list))

# --- Specs flattening (flatten all nested lists) ---
def flatten_specs_list(specs_data):
    all_specs = []
    if isinstance(specs_data, dict):
        for section in specs_data.values():
            if isinstance(section, list):
                all_specs.extend(section)
    return {spec.get('key'): spec.get('value') for spec in all_specs if isinstance(spec, dict) and 'key' in spec and 'value' in spec}

df_banglore_specs = pd.json_normalize(df["new_car_specs"].apply(flatten_specs_list))

# --- Combine all sections ---
df_final = pd.concat([df_banglore_detail, df_banglore_overview, df_banglore_feature, df_banglore_specs], axis=1)

# --- Add city only once ---
df_final["City"] = "bangalore"

# --- Save to Excel ---
df_final.to_excel("structured_bangalore_cars.xlsx", index=False)

print("✅ Data has been structured and saved to 'structured_bangalore_cars.xlsx'")


✅ Data has been structured and saved to 'structured_bangalore_cars.xlsx'


Perfectly Done unstructured to structured Banglore cares

In [13]:
import pandas as pd
import ast
from pandas import json_normalize

# Load the Excel file
file_path = "csv_files/bangalore_cars.xlsx"
df = pd.read_excel(file_path, sheet_name="bangalore_cars.csv")

# Convert stringified dictionaries to actual dicts
def parse_dict_column(column):
    return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

df["new_car_detail"] = parse_dict_column(df["new_car_detail"])
df["new_car_overview"] = parse_dict_column(df["new_car_overview"])
df["new_car_feature"] = parse_dict_column(df["new_car_feature"])
df["new_car_specs"] = parse_dict_column(df["new_car_specs"])

# ------------------------
# OVERVIEW
# ------------------------
def flatten_overview(overview_data):
    if isinstance(overview_data, dict):
        return {item['key']: item['value'] for item in overview_data.get("top", []) if isinstance(item, dict)}
    return {}

df_banglore_overview = pd.json_normalize(df["new_car_overview"].apply(flatten_overview))

# ------------------------
# DETAIL
# ------------------------
df_banglore_detail = json_normalize(df["new_car_detail"])

# ------------------------
# FEATURE: top + data[*].list
# ------------------------
def flatten_feature_all(data):
    features = []
    
    # Collect top values
    if isinstance(data, dict):
        top = data.get("top", [])
        features.extend(f.get("value") for f in top if isinstance(f, dict))
        
        # Collect from data section
        for section in data.get("data", []):
            if isinstance(section, dict):
                for item in section.get("list", []):
                    if isinstance(item, dict):
                        val = item.get("value")
                        if val:
                            features.append(val)

    # Return as key-value Feature_0, Feature_1...
    return {'Feature_' + str(i): v for i, v in enumerate(features)}

df_banglore_feature = pd.json_normalize(df["new_car_feature"].apply(flatten_feature_all))

# ------------------------
# SPECS: top + data[*].list (key-value pairs)
# ------------------------
def flatten_specs_all(data):
    specs = {}

    if isinstance(data, dict):
        # Top section
        for item in data.get("top", []):
            if isinstance(item, dict):
                k = item.get("key")
                v = item.get("value")
                if k and v:
                    specs[k] = v

        # From data -> list
        for section in data.get("data", []):
            if isinstance(section, dict):
                for item in section.get("list", []):
                    if isinstance(item, dict):
                        k = item.get("key")
                        v = item.get("value")
                        if k and v:
                            specs[k] = v
    return specs

df_banglore_specs = pd.json_normalize(df["new_car_specs"].apply(flatten_specs_all))

# ------------------------
# COMBINE ALL & ADD CITY ONCE
# ------------------------
df_final = pd.concat([df_banglore_detail, df_banglore_overview, df_banglore_feature, df_banglore_specs], axis=1)
df_final["City"] = "bangalore"

# ------------------------
# SAVE TO EXCEL
# ------------------------
df_final.to_excel("structured_bangalore_cars.xlsx", index=False)

print("✅ All nested features and specs parsed successfully and saved to 'structured_bangalore_cars.xlsx'")


✅ All nested features and specs parsed successfully and saved to 'structured_bangalore_cars.xlsx'


In [21]:
import pandas as pd

file_path = r"C:\Users\USER\Desktop\Car-Dheko_Used_Car_Price_Prediction\structured_bangalore_cars.xlsx"

# Get all sheet names
sheets = pd.ExcelFile(file_path).sheet_names
print("Available sheets:", sheets)


Available sheets: ['Sheet1']


In [22]:
df1 = pd.read_excel(file_path, sheet_name="Sheet1")  # or sheet_name=0 for the first sheet
print(df1.head())

   it      ft         bt        km transmission  ownerNo      owner      oem  \
0   0  Petrol  Hatchback  1,20,000       Manual        3  3rd Owner   Maruti   
1   0  Petrol        SUV    32,706       Manual        2  2nd Owner     Ford   
2   0  Petrol  Hatchback    11,949       Manual        1  1st Owner     Tata   
3   0  Petrol      Sedan    17,794       Manual        1  1st Owner  Hyundai   
4   0  Diesel        SUV    60,000       Manual        1  1st Owner   Maruti   

                model  modelYear  centralVariantId               variantName  \
0      Maruti Celerio       2015              3979                       VXI   
1       Ford Ecosport       2018              6087  1.5 Petrol Titanium BSIV   
2          Tata Tiago       2018              2983           1.2 Revotron XZ   
3       Hyundai Xcent       2014              1867        1.2 Kappa S Option   
4  Maruti SX4 S Cross       2015              4277             DDiS 200 Zeta   

         price priceActual  priceSavin

In [23]:
df1.transmission.unique()

array(['Manual', 'Automatic'], dtype=object)

all unique values check of banglore cars

In [24]:

# For all columns
for col in df1.columns:
    print(f"Column: {col}")
    print(df1[col].unique())
    print()  # Adds a blank line between columns

Column: it
[0]

Column: ft
['Petrol' 'Diesel' 'Lpg' 'Cng' 'Electric']

Column: bt
['Hatchback' 'SUV' 'Sedan' 'MUV' 'Coupe' 'Minivans' 'Pickup Trucks'
 'Convertibles']

Column: km
['1,20,000' '32,706' '11,949' '17,794' '60,000' '20,000' '37,772' '30,000'
 '37,000' '50,000' '24,887' '23,000' '48,928' '1,08,862' '80,000' '16,000'
 '65,376' '10,000' '67,000' '1,00,000' '18,083' '57,634' '1,10,000'
 '23,447' '33,000' '87,667' '40,000' '79,012' '31,344' '12,434' '12,000'
 '40,987' '18,157' '89,667' '56,997' '59,000' '18,835' '60,932' '33,539'
 '66,837' '400' '44,285' '25,000' '36,735' '79,000' '53,645' '53,982'
 '37,741' '17,367' '94,000' '33,700' '54,601' '64,000' '40,525' '44,154'
 '70,000' '68,039' '49,873' '90,000' '48,960' '68,000' '23,017' '83,000'
 '92,000' '62,139' '74,000' '26,500' '81,716' '24,000' '18,000' '38,197'
 '19,000' '14,000' '29,268' '64,374' '7,100' '43,679' '40,562' '58,600'
 '42,050' '65,017' '36,349' '83,394' '52,204' '89,000' '29,813' '13,345'
 '17,100' '75,000' '75,

In [25]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481 entries, 0 to 1480
Columns: 207 entries, it to City
dtypes: float64(9), int64(4), object(194)
memory usage: 2.3+ MB


In [35]:
# Shows all columns and their dtypes
with pd.option_context('display.max_rows', None):
    print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481 entries, 0 to 1480
Columns: 207 entries, it to City
dtypes: Int64(13), string(194)
memory usage: 2.4 MB
None


Custom NaN Summary (Best for Large DataFrames)
This creates a table showing missing values (NaN/None) for every column:

In [36]:
# Get NaN counts and dtypes for all columns
nan_summary = pd.DataFrame({
    'Column': df1.columns,
    'Data Type': df1.dtypes,
    'NaN Count': df1.isna().sum(),
    'NaN Percentage': (df1.isna().mean() * 100).round(2)
})

# Display ALL rows without truncation
with pd.option_context('display.max_rows', None, 'display.width', 1000):
    print(nan_summary)

                                            Column       Data Type  NaN Count  NaN Percentage
it                                              it           Int64          0            0.00
ft                                              ft  string[python]          0            0.00
bt                                              bt  string[python]          0            0.00
km                                              km  string[python]          0            0.00
transmission                          transmission  string[python]          0            0.00
ownerNo                                    ownerNo           Int64          0            0.00
owner                                        owner  string[python]          0            0.00
oem                                            oem  string[python]          0            0.00
model                                        model  string[python]          0            0.00
modelYear                                modelYear          

In [37]:
import pandas as pd
import numpy as np
import re

def clean_and_standardize(df):
    """Complete data cleaning pipeline with:
    1. Automatic dtype conversion
    2. Smart missing value imputation
    3. Unit standardization
    4. Format enforcement"""
    
    # ===== 1. Initial Type Conversion =====
    df = df.convert_dtypes()
    
    # ===== 2. Standardize Units & Clean Strings =====
    unit_patterns = {
        'km': r'(\d+)\s*kms?',
        'kg': r'(\d+)\s*kgs?',
        'years': r'(\d+)\s*years?',
        'percent': r'(\d+)%'
    }
    
    for col in df.columns:
        # ---- Handle String Columns ----
        if pd.api.types.is_string_dtype(df[col]):
            # Remove extra whitespace
            df[col] = df[col].str.strip()
            
            # Check for numeric values with units
            for unit, pattern in unit_patterns.items():
                if df[col].str.contains(pattern, regex=True, na=False).any():
                    # Extract numeric values
                    df[col] = df[col].str.extract(pattern, expand=False)
                    try:
                        df[col] = pd.to_numeric(df[col])
                        print(f"Converted {col} from string+unit to numeric")
                        break
                    except:
                        continue
        
        # ---- Convert Boolean-like Strings ----
        if df[col].nunique() == 2:
            if set(df[col].dropna().unique()) == {'Yes', 'No'}:
                df[col] = df[col].map({'Yes': True, 'No': False})
    
    # ===== 3. Handle Missing Values =====
    for col in df.columns:
        # Numeric columns
        if pd.api.types.is_numeric_dtype(df[col]):
            if df[col].isna().any():
                df[col] = df[col].fillna(df[col].median())
                
        # Categorical columns
        elif pd.api.types.is_string_dtype(df[col]) or pd.api.types.is_categorical_dtype(df[col]):
            if df[col].isna().any():
                if df[col].nunique() < 20:
                    df[col] = df[col].fillna(df[col].mode()[0])
                else:
                    df[col] = df[col].cat.add_categories(['Missing']).fillna('Missing')
    
    # ===== 4. Final Type Optimization =====
    df = df.convert_dtypes()
    
    return df

# Apply to your DataFrame
df1_clean = clean_and_standardize(df1)

# ===== Verification =====
def get_data_profile(df):
    profile = pd.DataFrame({
        'dtype': df.dtypes,
        'unique_values': df.nunique(),
        'na_count': df.isna().sum(),
        'sample_values': df.iloc[0].astype(str) + "..." + df.iloc[-1].astype(str)
    })
    return profile

with pd.option_context('display.max_rows', None):
    display(get_data_profile(df1_clean))

C:\Users\USER\AppData\Local\Temp\ipykernel_20216\2148212616.py:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if df[col].str.contains(pattern, regex=True, na=False).any():


Converted Mileage from string+unit to numeric
Converted Torque from string+unit to numeric
Converted Max Torque from string+unit to numeric
Converted Kerb Weight from string+unit to numeric
Converted Gross Weight from string+unit to numeric
Converted Top Speed from string+unit to numeric


AttributeError: Can only use .cat accessor with a 'category' dtype

In [38]:
def clean_and_standardize(df):
    """Improved pipeline with:
    - No regex warnings
    - Better unit handling
    - Progress tracking"""
    
    # ===== 1. Initial Type Conversion =====
    df = df.convert_dtypes()
    
    # ===== 2. Unit Standardization =====
    unit_conversions = {
        'distance': (r'(\d+)\s*(kms?|miles?)', lambda x: int(x)),
        'weight': (r'(\d+)\s*(kgs?|kilograms?)', lambda x: int(x)),
        'percentage': (r'(\d+)\s*%', lambda x: int(x)),
        'torque': (r'(\d+)\s*(nm|n·m)', lambda x: int(x)),
    }
    
    converted_columns = []
    
    for col in df.columns:
        if pd.api.types.is_string_dtype(df[col]):
            # ---- Clean whitespace ----
            df[col] = df[col].str.strip()
            
            # ---- Try all unit patterns ----
            for unit_type, (pattern, converter) in unit_conversions.items():
                # Extract values using regex
                extracted = df[col].str.extract(pattern, flags=re.IGNORECASE)[0]
                
                # If matches found, convert
                if not extracted.isna().all():
                    df[col] = extracted.dropna().apply(converter)
                    converted_columns.append((col, unit_type))
                    break
    
    # ===== 3. Missing Value Handling =====
    # ... (rest of your existing code) ...
    
    # Print conversion report
    print("\n=== Unit Conversion Report ===")
    for col, unit_type in converted_columns:
        print(f"Converted {col} ({unit_type}) to numeric")
    
    return df

work33333

In [ ]:
import pandas as pd
import re

def clean_and_standardize(df):
    """Pipeline to:
    - Convert units to numbers
    - Handle NaNs more effectively
    - Report changes
    """

    # ===== 1. Initial Type Conversion =====
    df = df.copy()
    df = df.convert_dtypes()

    # ===== 2. Unit Standardization =====
    unit_conversions = {
        'distance': (r'(\d+(?:\.\d+)?)\s*(kms?|miles?)', float),
        'weight': (r'(\d+(?:\.\d+)?)\s*(kgs?|kilograms?)', float),
        'percentage': (r'(\d+(?:\.\d+)?)\s*%', float),
        'torque': (r'(\d+(?:\.\d+)?)\s*(nm|n·m)', float),
    }

    converted_columns = []

    for col in df.columns:
        if pd.api.types.is_string_dtype(df[col]):
            df[col] = df[col].str.strip()

            for unit_type, (pattern, converter) in unit_conversions.items():
                extracted = df[col].str.extract(pattern, flags=re.IGNORECASE)[0]
                
                # If there are any values extracted, we replace and convert
                if extracted.notna().sum() > 0:
                    df[col] = pd.to_numeric(extracted, errors='coerce')
                    converted_columns.append((col, unit_type))
                    break  # Stop after first matching unit pattern

    # ===== 3. Impute Missing Values =====
    for col in df.columns:
        if df[col].isna().sum() == 0:
            continue  # Skip columns with no missing values

        if pd.api.types.is_numeric_dtype(df[col]):
            # Use median for numeric columns (more robust than mean)
            df[col].fillna(df[col].median(), inplace=True)
        else:
            # Use mode for categorical/string columns
            mode = df[col].mode()
            if not mode.empty:
                df[col].fillna(mode[0], inplace=True)
            else:
                df[col].fillna("Unknown", inplace=True)

    # ===== 4. Report =====
    print("\n=== Unit Conversion Report ===")
    for col, unit_type in converted_columns:
        print(f"Converted {col} ({unit_type}) to numeric")

    # Final report on missing values
    nan_stats = df.isna().sum()
    if nan_stats.any():
        print("\n⚠️ Still missing values in some columns:")
        print(nan_stats[nan_stats > 0])
    else:
        print("\n✅ No missing values remain.")

    return df


In [41]:
import pandas as pd

nan_summary = pd.DataFrame({
    'Column': df1.columns,
    'Data Type': df1.dtypes,
    'Non-Null Count': df1.notna().sum(),
    'NaN Count': df1.isna().sum(),
    'NaN Percentage': (df1.isna().mean() * 100).round(2)
}).reset_index(drop=True)

# Display ALL rows without truncation
with pd.option_context('display.max_rows', None, 'display.width', 1000):
    print(nan_summary)


                       Column       Data Type  Non-Null Count  NaN Count  NaN Percentage
0                          it           Int64            1481          0            0.00
1                          ft  string[python]            1481          0            0.00
2                          bt  string[python]            1481          0            0.00
3                          km  string[python]            1481          0            0.00
4                transmission  string[python]            1481          0            0.00
5                     ownerNo           Int64            1481          0            0.00
6                       owner  string[python]            1481          0            0.00
7                         oem  string[python]            1481          0            0.00
8                       model  string[python]            1481          0            0.00
9                   modelYear           Int64            1481          0            0.00
10           centralV

In [26]:
# Shows all columns and their dtypes
with pd.option_context('display.max_rows', None):
    print(df1.dtypes)


it                            int64
ft                           object
bt                           object
km                           object
transmission                 object
ownerNo                       int64
owner                        object
oem                          object
model                        object
modelYear                     int64
centralVariantId              int64
variantName                  object
price                        object
priceActual                  object
priceSaving                 float64
priceFixedText              float64
trendingText.imgUrl          object
trendingText.heading         object
trendingText.desc            object
Registration Year            object
Insurance Validity           object
Fuel Type                    object
Seats                        object
Kms Driven                   object
RTO                          object
Ownership                    object
Engine Displacement          object
Transmission                

In [27]:
with pd.option_context('display.max_rows', None):
    print(df1.dtypes)


it                            int64
ft                           object
bt                           object
km                           object
transmission                 object
ownerNo                       int64
owner                        object
oem                          object
model                        object
modelYear                     int64
centralVariantId              int64
variantName                  object
price                        object
priceActual                  object
priceSaving                 float64
priceFixedText              float64
trendingText.imgUrl          object
trendingText.heading         object
trendingText.desc            object
Registration Year            object
Insurance Validity           object
Fuel Type                    object
Seats                        object
Kms Driven                   object
RTO                          object
Ownership                    object
Engine Displacement          object
Transmission                

In [29]:
df1 = df1.convert_dtypes()  # Automatically converts object → string, numbers → Int64/Float64
print(df1.dtypes)

it                                   Int64
ft                          string[python]
bt                          string[python]
km                          string[python]
transmission                string[python]
                                 ...      
Cargo Volumn                string[python]
Wheel Size                  string[python]
Alloy Wheel Size            string[python]
Ground Clearance Unladen    string[python]
City                        string[python]
Length: 207, dtype: object


In [30]:
with pd.option_context('display.max_rows', None):
    print(df1.dtypes)


it                                   Int64
ft                          string[python]
bt                          string[python]
km                          string[python]
transmission                string[python]
ownerNo                              Int64
owner                       string[python]
oem                         string[python]
model                       string[python]
modelYear                            Int64
centralVariantId                     Int64
variantName                 string[python]
price                       string[python]
priceActual                 string[python]
priceSaving                          Int64
priceFixedText                       Int64
trendingText.imgUrl         string[python]
trendingText.heading        string[python]
trendingText.desc           string[python]
Registration Year           string[python]
Insurance Validity          string[python]
Fuel Type                   string[python]
Seats                       string[python]
Kms Driven 

Clean and Standardize Formats

To clean values like "70 kms" or "20000 INR" and convert them to numeric, we can use regex on all string columns:

In [33]:
import pandas as pd
import re

def clean_numeric_strings(df1):
    for col in df1.select_dtypes(include='object').columns:
        df1[col] = df1[col].str.replace(r'[^\d.\-]', '', regex=True)
        # Try converting to numeric if it makes sense
        try:
            df1[col] = pd.to_numeric(df1[col])
        except:
            continue
    return df1


In [34]:
with pd.option_context('display.max_rows', None):
    print(df1.dtypes)


it                                   Int64
ft                          string[python]
bt                          string[python]
km                          string[python]
transmission                string[python]
ownerNo                              Int64
owner                       string[python]
oem                         string[python]
model                       string[python]
modelYear                            Int64
centralVariantId                     Int64
variantName                 string[python]
price                       string[python]
priceActual                 string[python]
priceSaving                          Int64
priceFixedText                       Int64
trendingText.imgUrl         string[python]
trendingText.heading        string[python]
trendingText.desc           string[python]
Registration Year           string[python]
Insurance Validity          string[python]
Fuel Type                   string[python]
Seats                       string[python]
Kms Driven 

Handle Missing Values
🔹 For categorical columns:
Use mode imputation:

In [ ]:
for col in df1.select_dtypes(include='object').columns:
    df1[col].fillna(df1[col].mode()[0], inplace=True)

🔹 For numerical columns:
Use median imputation or optionally create a “missing” category:

In [ ]:
for col in df1.select_dtypes(include=['int64', 'float64']).columns:
    df1[col].fillna(df1[col].median(), inplace=True)


In [8]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)      # Show more rows if needed
print(df_final.head(3))                     # Preview the data


NameError: name 'df_final' is not defined

In [7]:
print(df_final.columns.tolist())


NameError: name 'df_final' is not defined

In [2]:
df1.it.unique()

NameError: name 'df1' is not defined

In [1]:
import pandas as pd
import os
from glob import glob

# 1. Specify the correct folder path (where your Excel files are stored)
folder_path = "csv_files"  # Replace with your actual folder name if different

# 2. List all city Excel files in the folder
city_files = glob(os.path.join(folder_path, '*.xlsx'))  # Searches inside 'csv_files' folder
print(f"Found Excel files in '{folder_path}':", city_files)

if not city_files:
    print(f"\n❌ No Excel files found in '{folder_path}'. Please check:")
    print("- Folder name is correct (case-sensitive)")
    print("- Files have '.xlsx' extension")
    print("- Files are not in a sub-subfolder")
else:
    # 3. Create a summary DataFrame
    summary_data = []

    for file in city_files:
        city_name = os.path.splitext(os.path.basename(file))[0]  # Extract filename without path/extension
        try:
            # Read first 2 rows to check structure
            df_sample = pd.read_excel(file, nrows=2)
            
            # Get file info
            file_size = f"{round(os.path.getsize(file)/(1024*1024), 2)} MB"
            file_columns = df_sample.columns.tolist()
            file_shape = pd.read_excel(file).shape
            
            summary_data.append({
                'City': city_name,
                'File_Type': 'Excel (XLSX)',
                'File_Size': file_size,
                'Rows': file_shape[0],
                'Columns': file_shape[1],
                'Columns_List': file_columns,
                'Sample_First_Row': df_sample.iloc[0].to_dict() if len(df_sample) > 0 else {}
            })
        except Exception as e:
            print(f"Error reading {file}: {str(e)}")
            summary_data.append({
                'City': city_name,
                'Error': str(e)
            })

    # 4. Create summary DataFrame
    summary_df = pd.DataFrame(summary_data)

    # 5. Display results
    if not summary_df.empty:
        print("\n📊 File Summary:")
        print(summary_df[['City', 'File_Type', 'File_Size', 'Rows', 'Columns']])
        
        # Save to Excel
        output_path = os.path.join(folder_path, 'city_files_summary.xlsx')
        summary_df.to_excel(output_path, index=False)
        print(f"\n✅ Summary saved to: {output_path}")
    else:
        print("\n❌ All files failed to load. Check error messages above.")

Found Excel files in 'csv_files': ['csv_files\\bangalore_cars.xlsx', 'csv_files\\chennai_cars.xlsx', 'csv_files\\city_files_summary.xlsx', 'csv_files\\delhi_cars.xlsx', 'csv_files\\hyderabad_cars.xlsx', 'csv_files\\jaipur_cars.xlsx', 'csv_files\\kolkata_cars.xlsx']

📊 File Summary:
                 City     File_Type File_Size  Rows  Columns
0      bangalore_cars  Excel (XLSX)   0.55 MB  1481        5
1        chennai_cars  Excel (XLSX)   0.54 MB  1419        5
2  city_files_summary  Excel (XLSX)   0.01 MB     6        7
3          delhi_cars  Excel (XLSX)   0.56 MB  1485        5
4      hyderabad_cars  Excel (XLSX)   0.55 MB  1483        5
5         jaipur_cars  Excel (XLSX)    0.4 MB  1120        5
6        kolkata_cars  Excel (XLSX)   0.51 MB  1381        5

✅ Summary saved to: csv_files\city_files_summary.xlsx


In [2]:
# Test reading one file
test_file = city_files[0]  # or specify a filename
try:
    test_df = pd.read_excel(test_file)
    print(f"Successfully read {test_file}")
    print("Columns:", test_df.columns.tolist())
except Exception as e:
    print(f"Failed to read {test_file}: {str(e)}")

Successfully read csv_files\bangalore_cars.xlsx
Columns: ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs', 'car_links']


In [3]:
# Enhanced Version (Checks All Sheets):
# If your Excel files have multiple sheets, use this:

# Modified loop to check all sheets
for file in city_files:
    city_name = os.path.splitext(file)[0]
    try:
        xl = pd.ExcelFile(file)  # Open Excel file
        for sheet_name in xl.sheet_names:  # Loop through all sheets
            df_sample = xl.parse(sheet_name, nrows=2)
            file_size = f"{round(os.path.getsize(file)/(1024*1024), 2)} MB"
            summary_data.append({
                'City': city_name,
                'Sheet_Name': sheet_name,
                'File_Type': 'Excel (XLSX)',
                'File_Size': file_size,
                'Rows': xl.parse(sheet_name).shape[0],
                'Columns': xl.parse(sheet_name).shape[1],
                'Columns_List': df_sample.columns.tolist(),
                'Dtypes': df_sample.dtypes.to_dict()
            })
    except Exception as e:
        print(f"Error reading {file}: {str(e)}")

In [4]:

# Check for consistency across files:
# Compare columns across all cities
all_columns = summary_df['Columns_List'].explode().unique()
print("Unique columns across all files:", all_columns)

Unique columns across all files: ['new_car_detail' 'new_car_overview' 'new_car_feature' 'new_car_specs'
 'car_links' 'City' 'File_Type' 'File_Size' 'Rows' 'Columns'
 'Columns_List' 'Sample_First_Row']


In [5]:
# Merge all files into a single DataFrame:

all_cities = pd.concat(
    [pd.read_excel(file).assign(City=os.path.splitext(file)[0]) 
    for file in city_files
])

In [ ]:
#Reset the Index
all_cities = all_cities.reset_index(drop=True)

In [9]:
#Check for Missing Values
print(all_cities.isnull().sum())

new_car_detail         7
new_car_overview       7
new_car_feature        7
new_car_specs          7
car_links              7
City                   0
File_Type           8369
File_Size           8369
Rows                8369
Columns             8369
Columns_List        8369
Sample_First_Row    8369
dtype: int64


In [10]:
print(all_cities.head())
print(all_cities.info())

                                      new_car_detail  \
0  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
1  {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...   
2  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
3  {'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...   
4  {'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...   

                                    new_car_overview  \
0  {'heading': 'Car overview', 'top': [{'key': 'R...   
1  {'heading': 'Car overview', 'top': [{'key': 'R...   
2  {'heading': 'Car overview', 'top': [{'key': 'R...   
3  {'heading': 'Car overview', 'top': [{'key': 'R...   
4  {'heading': 'Car overview', 'top': [{'key': 'R...   

                                     new_car_feature  \
0  {'heading': 'Features', 'top': [{'value': 'Pow...   
1  {'heading': 'Features', 'top': [{'value': 'Pow...   
2  {'heading': 'Features', 'top': [{'value': 'Pow...   
3  {'heading': 'Features', 'top': [{'value': 'Pow...   
4  {'heading': 'Features', 'top': [{'value': '

In [ ]:
all_cities.columns = all_cities.columns.str.strip().str.lower().str.replace(' ', '_')
#Makes column names consistent.

In [13]:
#Handle Data Type Issues  Convert columns to appropriate data types.
all_cities['date_column'] = pd.to_datetime(all_cities['date_column'])

KeyError: 'date_column'

In [14]:
print(all_cities.columns.tolist())

['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs', 'car_links', 'city', 'file_type', 'file_size', 'rows', 'columns', 'columns_list', 'sample_first_row']


In [17]:
all_cities['date'] = pd.to_datetime(all_cities['date'], errors='coerce')  # converts errors to NaT

KeyError: 'date'

In [ ]:
import pandas as pd
import ast
import re
from decimal import Decimal
from pandas import json_normalize

# ------------------------
# LOAD AND PARSE DATA
# ------------------------

file_path = "csv_files/bangalore_cars.xlsx"
df = pd.read_excel(file_path, sheet_name="bangalore_cars.csv")

def parse_dict_column(column):
    return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

df["new_car_detail"] = parse_dict_column(df["new_car_detail"])
df["new_car_overview"] = parse_dict_column(df["new_car_overview"])
df["new_car_feature"] = parse_dict_column(df["new_car_feature"])
df["new_car_specs"] = parse_dict_column(df["new_car_specs"])

# ------------------------
# FLATTEN DATA
# ------------------------

def flatten_overview(data):
    if isinstance(data, dict):
        return {item["key"]: item["value"] for item in data.get("top", []) if isinstance(item, dict)}
    return {}

df_banglore_overview = json_normalize(df["new_car_overview"].apply(flatten_overview))
df_banglore_detail = json_normalize(df["new_car_detail"])

def flatten_feature_all(data):
    features = []
    if isinstance(data, dict):
        features.extend(f.get("value") for f in data.get("top", []) if isinstance(f, dict))
        for section in data.get("data", []):
            for item in section.get("list", []):
                if isinstance(item, dict):
                    val = item.get("value")
                    if val:
                        features.append(val)
    return {"Feature_" + str(i): v for i, v in enumerate(features)}

df_banglore_feature = json_normalize(df["new_car_feature"].apply(flatten_feature_all))

def flatten_specs_all(data):
    specs = {}
    if isinstance(data, dict):
        for item in data.get("top", []):
            if isinstance(item, dict):
                specs[item.get("key")] = item.get("value")
        for section in data.get("data", []):
            for item in section.get("list", []):
                if isinstance(item, dict):
                    specs[item.get("key")] = item.get("value")
    return specs

df_banglore_specs = json_normalize(df["new_car_specs"].apply(flatten_specs_all))

# COMBINE ALL
df_final = pd.concat([df_banglore_detail, df_banglore_overview, df_banglore_feature, df_banglore_specs], axis=1)
df_final["City"] = "bangalore"

# ------------------------
# CLEANING AND TRANSFORMATION
# ------------------------

def clean_and_transform(df):
    df = df.copy()

    def extract_price_in_rupees(price_str):
        match = re.search(r'₹\s*(\d+(\.\d+)?)\s*Lakh', str(price_str))
        if match:
            return float(Decimal(match.group(1)) * 100000)
        return None

    df['price_in_rupees'] = df['price'].apply(extract_price_in_rupees) if 'price' in df.columns else None

    to_datetime_cols = {
        'Registration Year': '%Y',
        'modelYear': '%Y'
    }
    for col, fmt in to_datetime_cols.items():
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format=fmt, errors='coerce').dt.year

    def extract_int(col, pattern=r'(\d+)'):
        return df[col].str.extract(pattern).astype('Int64') if col in df.columns else None

    string_to_int_cols = [
        'Seats', 'Kms Driven', 'Engine Displacement', 'Displacement', 
        'Length', 'Width', 'Height', 'Rear Tread', 'Kerb Weight',
        'Cargo Volumn', 'Top Speed', 'No Door Numbers', 'Seating Capacity',
        'Ground Clearance Unladen'
    ]
    for col in string_to_int_cols:
        if col in df.columns:
            df[col] = extract_int(col)

    if 'Mileage' in df.columns:
        df['Mileage'] = df['Mileage'].str.extract(r'(\d+\.?\d*)').astype(float)

    if 'Max Power' in df.columns:
        power_split = df['Max Power'].str.extract(r'(\d+\.?\d*)bhp@(\d+)rpm')
        df['BHP'] = power_split[0].astype(float)
        df['RPM'] = power_split[1].astype('Int64')

    if 'Torque' in df.columns:
        df['Torque'] = extract_int('Torque')

    if 'Gear Box' in df.columns:
        df['gear_box_number'] = extract_int('Gear Box')

    if 'Turning Radius' in df.columns:
        df['Turning Radius'] = df['Turning Radius'].str.extract(r'(\d+\.?\d*)').astype(float)

    if 'Wheel Size' in df.columns:
        df['Wheel Size'] = extract_int('Wheel Size', r'R(\d+)')

    if 'Alloy Wheel Size' in df.columns:
        df['Alloy Wheel Size'] = extract_int('Alloy Wheel Size', r'R(\d+)')

    if 'Year of Manufacture' in df.columns:
        df['Year of Manufacture'] = df['Year of Manufacture'].astype(str).str.replace(r'\.', '', regex=True).astype('Int64')

    return df

df_final_cleaned = clean_and_transform(df_final)

# ------------------------
# SAVE TO EXCEL
# ------------------------

df_final_cleaned.to_excel("structured_c_bangalore_cars.xlsx", index=False)
print("✅ Data parsed, cleaned, and saved to 'structured_c_bangalore_cars.xlsx'")

# ------------------------
# DATA QUALITY REPORT
# ------------------------

nan_summary = pd.DataFrame({
    'Column': df_final_cleaned.columns,
    'Data Type': df_final_cleaned.dtypes,
    'NaN Count': df_final_cleaned.isna().sum(),
    'NaN Percentage': (df_final_cleaned.isna().mean() * 100).round(2)
})

with pd.option_context('display.max_rows', None, 'display.width', 1000):
    print("\n=== Data Quality Report ===")
    print(nan_summary)


BANGLORE CLEANED DATA SET

In [5]:
import pandas as pd
import ast
import re
from decimal import Decimal
from pandas import json_normalize

# ------------------------
# LOAD AND PARSE DATA
# ------------------------

file_path = "csv_files/bangalore_cars.xlsx"
df = pd.read_excel(file_path, sheet_name="bangalore_cars.csv")

def parse_dict_column(column):
    return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

df["new_car_detail"] = parse_dict_column(df["new_car_detail"])
df["new_car_overview"] = parse_dict_column(df["new_car_overview"])
df["new_car_feature"] = parse_dict_column(df["new_car_feature"])
df["new_car_specs"] = parse_dict_column(df["new_car_specs"])

# ------------------------
# FLATTEN DATA
# ------------------------

def flatten_overview(data):
    if isinstance(data, dict):
        return {item["key"]: item["value"] for item in data.get("top", []) if isinstance(item, dict)}
    return {}

df_banglore_overview = json_normalize(df["new_car_overview"].apply(flatten_overview))
df_banglore_detail = json_normalize(df["new_car_detail"])

def flatten_feature_all(data):
    features = []
    if isinstance(data, dict):
        features.extend(f.get("value") for f in data.get("top", []) if isinstance(f, dict))
        for section in data.get("data", []):
            for item in section.get("list", []):
                if isinstance(item, dict):
                    val = item.get("value")
                    if val:
                        features.append(val)
    return {"Feature_" + str(i): v for i, v in enumerate(features)}

df_banglore_feature = json_normalize(df["new_car_feature"].apply(flatten_feature_all))

def flatten_specs_all(data):
    specs = {}
    if isinstance(data, dict):
        for item in data.get("top", []):
            if isinstance(item, dict):
                specs[item.get("key")] = item.get("value")
        for section in data.get("data", []):
            for item in section.get("list", []):
                if isinstance(item, dict):
                    specs[item.get("key")] = item.get("value")
    return specs

df_banglore_specs = json_normalize(df["new_car_specs"].apply(flatten_specs_all))

# COMBINE ALL
df_final = pd.concat([df_banglore_detail, df_banglore_overview, df_banglore_feature, df_banglore_specs], axis=1)
df_final["City"] = "bangalore"

# ------------------------
# CLEANING AND TRANSFORMATION
# ------------------------

def clean_and_transform(df):
    df = df.copy()

    def extract_price_in_rupees(price_str):
        match = re.search(r'₹\s*(\d+(\.\d+)?)\s*Lakh', str(price_str))
        if match:
            return float(Decimal(match.group(1)) * 100000)
        return None

    df['price_in_rupees'] = df['price'].apply(extract_price_in_rupees) if 'price' in df.columns else None

    to_datetime_cols = {
        'Registration Year': '%Y',
        'modelYear': '%Y'
    }
    for col, fmt in to_datetime_cols.items():
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format=fmt, errors='coerce').dt.year
    
    # def extract_int(col, pattern=r'(\d+)'):
    #     return df[col].str.extract(pattern).astype('Int64') if col in df.columns else None
    
    # def extract_int(col, pattern=r'(\d+)'):
    #     return df[col].astype(str).str.extract(pattern).astype('Int64') if col in df.columns else None
    
    def extract_int(col, pattern=r'(\d+)'):
        if col in df.columns:
            return df[col].apply(lambda x: str(x) if pd.notnull(x) else '').str.extract(pattern)[0].astype('Int64')
        return None



    string_to_int_cols = [
        'Seats', 'Kms Driven', 'Engine Displacement', 'Displacement', 
        'Length', 'Width', 'Height', 'Rear Tread', 'Kerb Weight',
        'Cargo Volumn', 'Top Speed', 'No Door Numbers', 'Seating Capacity',
        'Ground Clearance Unladen'
    ]
    for col in string_to_int_cols:
        if col in df.columns:
            df[col] = extract_int(col)

    if 'Mileage' in df.columns:
        df['Mileage'] = df['Mileage'].str.extract(r'(\d+\.?\d*)').astype(float)

    if 'Max Power' in df.columns:
        power_split = df['Max Power'].str.extract(r'(\d+\.?\d*)bhp@(\d+)rpm')
        df['BHP'] = power_split[0].astype(float)
        df['RPM'] = power_split[1].astype('Int64')

    if 'Torque' in df.columns:
        df['Torque'] = extract_int('Torque')

    if 'Gear Box' in df.columns:
        df['gear_box_number'] = extract_int('Gear Box')

    if 'Turning Radius' in df.columns:
        df['Turning Radius'] = df['Turning Radius'].str.extract(r'(\d+\.?\d*)').astype(float)

    if 'Wheel Size' in df.columns:
        df['Wheel Size'] = extract_int('Wheel Size', r'R(\d+)')

    if 'Alloy Wheel Size' in df.columns:
        df['Alloy Wheel Size'] = extract_int('Alloy Wheel Size', r'R(\d+)')

    if 'Year of Manufacture' in df.columns:
        df['Year of Manufacture'] = df['Year of Manufacture'].astype(str).str.replace(r'\.', '', regex=True).astype('Int64')

    return df

df_final_cleaned = clean_and_transform(df_final)


# ------------------------
# b) HANDLING MISSING VALUES
# ------------------------

# i) Numerical columns - fill missing with median
for col in df_final_cleaned.select_dtypes(include=['int64', 'float64']).columns:
    df_final_cleaned[col].fillna(df_final_cleaned[col].median(), inplace=True)

# ii) Categorical columns - fill missing with mode
for col in df_final_cleaned.select_dtypes(include='object').columns:
    if df_final_cleaned[col].isnull().any():
        mode_value = df_final_cleaned[col].mode()[0]
        df_final_cleaned[col].fillna(mode_value, inplace=True)


# Add 'City' column as string
df_final_cleaned["City"] = "bangalore"
df_final_cleaned["City"] = df_final_cleaned["City"].astype(str)

# Ensure 'City' column is the last one
cols = list(df_final_cleaned.columns)
cols.append(cols.pop(cols.index("City")))
df_final_cleaned = df_final_cleaned[cols]
# ------------------------
# SAVE TO EXCEL
# ------------------------


df_final_cleaned.to_excel("structured_cleaned_bangalore_cars.xlsx", index=False, sheet_name='bangalore_cars')

print("✅ Cleaned data with missing values handled and saved to 'structured_cleaned_bangalore_cars.xlsx'")

# ------------------------
# DATA QUALITY REPORT
# ------------------------

nan_summary = pd.DataFrame({
    'Column': df_final_cleaned.columns,
    'Data Type': df_final_cleaned.dtypes,
    'NaN Count': df_final_cleaned.isna().sum(),
    'NaN Percentage': (df_final_cleaned.isna().mean() * 100).round(2)
})

with pd.option_context('display.max_rows', None, 'display.width', 1000):
    print("\n=== Data Quality Report ===")
    print(nan_summary)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [6]:
import pandas as pd
import ast
import re
from pandas import json_normalize
from decimal import Decimal
import numpy as np

def load_and_parse_data(file_path):
    """Load and parse the Excel file with nested JSON data"""
    try:
        df = pd.read_excel(file_path, sheet_name="bangalore_cars.csv")
        
        # Convert stringified dictionaries to actual dicts
        def parse_dict_column(column):
            return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        
        for col in ["new_car_detail", "new_car_overview", "new_car_feature", "new_car_specs"]:
            if col in df.columns:
                df[col] = parse_dict_column(df[col])
        
        return df
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return None

def flatten_data(df):
    """Flatten all nested structures into a single DataFrame"""
    try:
        # OVERVIEW
        def flatten_overview(overview_data):
            if isinstance(overview_data, dict):
                return {item['key']: item['value'] for item in overview_data.get("top", []) if isinstance(item, dict)}
            return {}
        
        df_overview = pd.json_normalize(df["new_car_overview"].apply(flatten_overview)) if "new_car_overview" in df.columns else pd.DataFrame()

        # DETAIL
        df_detail = json_normalize(df["new_car_detail"]) if "new_car_detail" in df.columns else pd.DataFrame()

        # FEATURE
        def flatten_feature(data):
            features = []
            if isinstance(data, dict):
                top = data.get("top", [])
                features.extend(f.get("value") for f in top if isinstance(f, dict))
                for section in data.get("data", []):
                    if isinstance(section, dict):
                        for item in section.get("list", []):
                            if isinstance(item, dict):
                                val = item.get("value")
                                if val:
                                    features.append(val)
            return {'Feature_' + str(i): v for i, v in enumerate(features)}
        
        df_feature = pd.json_normalize(df["new_car_feature"].apply(flatten_feature)) if "new_car_feature" in df.columns else pd.DataFrame()

        # SPECS
        def flatten_specs(data):
            specs = {}
            if isinstance(data, dict):
                for item in data.get("top", []):
                    if isinstance(item, dict):
                        k = item.get("key")
                        v = item.get("value")
                        if k and v:
                            specs[k] = v
                for section in data.get("data", []):
                    if isinstance(section, dict):
                        for item in section.get("list", []):
                            if isinstance(item, dict):
                                k = item.get("key")
                                v = item.get("value")
                                if k and v:
                                    specs[k] = v
            return specs
        
        df_specs = pd.json_normalize(df["new_car_specs"].apply(flatten_specs)) if "new_car_specs" in df.columns else pd.DataFrame()

        # Combine all DataFrames
        df_final = pd.concat([df_detail, df_overview, df_feature, df_specs], axis=1)
        df_final["City"] = "bangalore"
        
        return df_final
    except Exception as e:
        print(f"❌ Error flattening data: {e}")
        return None

def clean_numeric_column(series, pattern=None, dtype=float):
    """Helper function to clean numeric columns"""
    try:
        if series.dtype == object:
            if pattern:
                extracted = series.str.extract(pattern)[0]
            else:
                # For cases where we just need to remove non-numeric characters
                extracted = series.astype(str).str.replace(r'[^\d]', '', regex=True)
            
            if dtype == 'Int64':
                return pd.to_numeric(extracted, errors='coerce').astype('Int64')
            return pd.to_numeric(extracted, errors='coerce')
        return series
    except Exception as e:
        print(f"⚠️ Warning cleaning column {series.name}: {e}")
        return series

def clean_and_transform(df):
    """Perform all cleaning and transformation operations"""
    try:
        df = df.copy()
        
        # ===== SPECIAL HANDLING FOR YEAR OF MANUFACTURE =====
        if 'Year of Manufacture' in df.columns:
            # Step 1: Remove all non-numeric characters including dots
            df['Year of Manufacture'] = df['Year of Manufacture'].astype(str).str.replace(r'[^\d]', '', regex=True)
            
            # Step 2: Convert to integer (handling empty strings)
            df['Year of Manufacture'] = pd.to_numeric(df['Year of Manufacture'], errors='coerce').astype('Int64')
            
            # Step 3: Convert to datetime (year only)
            df['Year of Manufacture'] = pd.to_datetime(df['Year of Manufacture'], format='%Y', errors='coerce')
        
        # ===== STANDARD TRANSFORMATIONS =====
        transformations = [
            # Price conversion
            ('price', r'₹\s*(\d+(\.\d+)?)\s*Lakh', lambda x: float(Decimal(x.group(1)) * Decimal('100000'))),
            
            # Date/time conversions
            ('Registration Year', None, lambda x: pd.to_datetime(x, errors='coerce').dt.year),
            ('modelYear', None, lambda x: pd.to_datetime(x, format='%Y', errors='coerce').dt.year),
            
            # Simple numeric extractions
            ('Seats', r'(\d+)', 'Int64'),
            ('Kms Driven', r'(\d+)', 'Int64'),
            ('Engine Displacement', r'(\d+)', 'Int64'),
            ('Torque', r'(\d+)', 'Int64'),
            ('Rear Tread', r'(\d+)', 'Int64'),
            ('Kerb Weight', r'(\d+)', 'Int64'),
            ('Top Speed', r'(\d+)', 'Int64'),
            ('Cargo Volume', r'(\d+)', 'Int64'),
            ('Ground Clearance Unladen', r'(\d+)', 'Int64'),
            
            # Float extractions
            ('Mileage', r'(\d+\.?\d*)', float),
            ('Turning Radius', r'(\d+\.?\d*)', float),
            
            # Special cases
            ('Displacement', None, lambda x: clean_numeric_column(x, dtype='Int64')),
            ('Seating Capacity', None, lambda x: clean_numeric_column(x, dtype='Int64')),
            ('No Door Numbers', None, lambda x: clean_numeric_column(x, dtype='Int64')),
            
            # Wheel sizes
            ('Wheel Size', r'R?(\d+)', 'Int64'),
            ('Alloy Wheel Size', r'R?(\d+)', 'Int64'),
            
            # Max Power split
            ('Max Power', r'(\d+\.?\d*)bhp@(\d+)rpm', 
             lambda x: (pd.Series(x[0].astype(float)), pd.Series(x[1].astype('Int64'))),
             ['BHP', 'RPM']),
            
            # Acceleration
            ('Acceleration', r'(\d+\.?\d*)', float, 'Acceleration_seconds')
        ]
        
        for col, pattern, action, *extra in transformations:
            if col in df.columns:
                try:
                    if callable(action):
                        if pattern:
                            match = df[col].str.extract(pattern)
                            if len(extra) > 0 and isinstance(extra[0], list):
                                # For columns that split into multiple columns
                                result = action(match)
                                for i, new_col in enumerate(extra[0]):
                                    df[new_col] = result[i]
                            else:
                                df[extra[0] if extra else col] = action(match)
                        else:
                            df[col] = action(df[col])
                    else:
                        df[col] = clean_numeric_column(df[col], pattern, action)
                except Exception as e:
                    print(f"⚠️ Warning processing column {col}: {e}")
        
        return df
    except Exception as e:
        print(f"❌ Error in transformation: {e}")
        return None

def generate_data_quality_report(df):
    """Generate comprehensive data quality report"""
    report = pd.DataFrame({
        'Column': df.columns,
        'Data Type': df.dtypes,
        'NaN Count': df.isna().sum(),
        'NaN Percentage': (df.isna().mean() * 100).round(2),
        'Unique Values': df.nunique(),
        'Sample Values': df.apply(lambda x: x.dropna().sample(1).values[0] if len(x.dropna()) > 0 else None)
    })
    
    print("\n=== DATA QUALITY REPORT ===")
    with pd.option_context('display.max_rows', None, 'display.width', 1000):
        print(report)
    
    return report

def main():
    # Load and parse data
    file_path = "csv_files/bangalore_cars.xlsx"
    df = load_and_parse_data(file_path)
    if df is None:
        return
    
    # Flatten data
    df_flat = flatten_data(df)
    if df_flat is None:
        return
    
    # Clean and transform
    df_clean = clean_and_transform(df_flat)
    if df_clean is None:
        return
    
    # Generate report
    report = generate_data_quality_report(df_clean)
    
    # Save to Excel
    try:
        df_clean.to_excel("structured_cleaned_bangalore_cars.xlsx", index=False, sheet_name='bangalore_cars')
        print("\n✅ All operations completed successfully!")
        print("Saved to 'structured_c_bangalore_cars.xlsx'")
    except Exception as e:
        print(f"❌ Error saving file: {e}")

if __name__ == "__main__":
    main()

⚠️ Warning processing column price: 'DataFrame' object has no attribute 'group'
⚠️ Warning processing column Seats: 'DataFrame' object has no attribute 'name'
⚠️ Warning processing column Mileage: float() argument must be a string or a real number, not 'DataFrame'
⚠️ Warning processing column Turning Radius: float() argument must be a string or a real number, not 'DataFrame'
⚠️ Warning processing column Acceleration: float() argument must be a string or a real number, not 'DataFrame'

=== DATA QUALITY REPORT ===
                                            Column       Data Type  NaN Count  NaN Percentage  Unique Values                                      Sample Values
it                                              it           int64          0            0.00              1                                                  0
ft                                              ft          object          0            0.00              5                                             Petrol
bt

In [18]:
import pandas as pd 
import ast
import re
from pandas import json_normalize
import numpy as np
import os

def load_and_parse_data(file_path):
    """Load and parse the Excel file with nested JSON data"""
    try:
        df = pd.read_excel(file_path)
        
        # Convert stringified dictionaries to actual dicts
        def parse_dict_column(column):
            return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('{') else x)
        
        for col in ["new_car_detail", "new_car_overview", "new_car_feature", "new_car_specs"]:
            if col in df.columns:
                df[col] = parse_dict_column(df[col])
        
        return df
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return None

def flatten_data(df):
    """Flatten all nested structures into a single DataFrame"""
    try:
        # OVERVIEW
        def flatten_overview(overview_data):
            if isinstance(overview_data, dict):
                return {item['key']: item['value'] for item in overview_data.get("top", []) if isinstance(item, dict)}
            return {}
        
        df_overview = pd.json_normalize(df["new_car_overview"].apply(flatten_overview)) if "new_car_overview" in df.columns else pd.DataFrame()

        # DETAIL
        df_detail = json_normalize(df["new_car_detail"]) if "new_car_detail" in df.columns else pd.DataFrame()

        # FEATURE
        def flatten_feature(data):
            features = []
            if isinstance(data, dict):
                top = data.get("top", [])
                features.extend(f.get("value") for f in top if isinstance(f, dict))
                for section in data.get("data", []):
                    if isinstance(section, dict):
                        for item in section.get("list", []):
                            if isinstance(item, dict):
                                val = item.get("value")
                                if val:
                                    features.append(val)
            return {'Feature_' + str(i): v for i, v in enumerate(features)}
        
        df_feature = pd.json_normalize(df["new_car_feature"].apply(flatten_feature)) if "new_car_feature" in df.columns else pd.DataFrame()

        # SPECS
        def flatten_specs(data):
            specs = {}
            if isinstance(data, dict):
                for item in data.get("top", []):
                    if isinstance(item, dict):
                        k = item.get("key")
                        v = item.get("value")
                        if k and v:
                            specs[k] = v
                for section in data.get("data", []):
                    if isinstance(section, dict):
                        for item in section.get("list", []):
                            if isinstance(item, dict):
                                k = item.get("key")
                                v = item.get("value")
                                if k and v:
                                    specs[k] = v
            return specs
        
        df_specs = pd.json_normalize(df["new_car_specs"].apply(flatten_specs)) if "new_car_specs" in df.columns else pd.DataFrame()

        # Combine all DataFrames
        df_final = pd.concat([df[['car_links']], df_detail, df_overview, df_feature, df_specs], axis=1)
        df_final["City"] = "bangalore"
        
        return df_final
    except Exception as e:
        print(f"❌ Error flattening data: {e}")
        return None

def clean_numeric_series(series, dtype=float, pattern=None):
    """Clean a pandas Series containing numeric values"""
    try:
        if not pd.api.types.is_string_dtype(series):
            series = series.astype(str)
            
        # Remove commas and extract numeric values
        cleaned = series.str.replace(',', '')
        
        if pattern:
            extracted = cleaned.str.extract(pattern)[0]
        else:
            if dtype == float:
                extracted = cleaned.str.extract(r'([\d\.]+)')[0]
            else:
                extracted = cleaned.str.extract(r'(\d+)')[0]
        
        # Convert to appropriate type
        if dtype == 'Int64':
            return pd.to_numeric(extracted, errors='coerce').astype('Int64')
        return pd.to_numeric(extracted, errors='coerce').astype(dtype)
    except Exception as e:
        print(f"⚠️ Warning cleaning numeric series: {e}")
        return series

def clean_and_transform(df):
    """Perform all cleaning and transformation operations"""
    try:
        df = df.copy()
        
        # ===== PROCESS KM COLUMN =====
        if 'km' in df.columns:
            df['km'] = clean_numeric_series(df['km'], 'Int64')
            print("✅ Processed 'km' column - removed commas and converted to integer")
        
        # ===== PROCESS MODEL YEAR =====
        if 'modelYear' in df.columns:
            # Extract year and month (handles formats like "2023-05" or "2023/05")
            year_extract = df['modelYear'].astype(str).str.extract(r'(?P<year>\d{4})[-/]?(?P<month>\d{2})?')
            
            if not year_extract.empty:
                df['modelYear_Year'] = clean_numeric_series(year_extract['year'], 'Int64')
                if 'month' in year_extract.columns:
                    df['modelYear_Month'] = clean_numeric_series(year_extract['month'], 'Int64')
                print("✅ Processed 'modelYear' - extracted year and month")
        
        # ===== PROCESS OTHER NUMERIC COLUMNS =====
        numeric_transforms = [
            ('Seats', 'Int64'),
            ('Mileage', float),
            ('Turning Radius', float),
            ('Acceleration', float),
            ('Kms Driven', 'Int64'),
            ('Engine Displacement', 'Int64'),
            ('Torque', 'Int64'),
            ('Rear Tread', 'Int64'),
            ('Kerb Weight', 'Int64'),
            ('Top Speed', 'Int64'),
            ('Cargo Volume', 'Int64'),
            ('Ground Clearance Unladen', 'Int64'),
            ('Displacement', 'Int64'),
            ('Seating Capacity', 'Int64'),
            ('No Door Numbers', 'Int64'),
            ('Wheel Size', 'Int64'),
            ('Alloy Wheel Size', 'Int64')
        ]
        
        for col, dtype in numeric_transforms:
            if col in df.columns:
                df[col] = clean_numeric_series(df[col], dtype)
                print(f"✅ Processed '{col}' - converted to {dtype}")
        
        # ===== PROCESS PRICE COLUMN =====
        if 'price' in df.columns:
            # Convert prices like "₹5.5 Lakh" to 550000
            df['price'] = (
                df['price']
                .astype(str)
                .str.replace(r'[^\d.]', '', regex=True)
                .astype(float)
                .mul(100000)
            )
            print("✅ Processed 'price' - converted to numeric INR")
        
        return df
    except Exception as e:
        print(f"❌ Error in transformation: {e}")
        return None

def save_cleaned_data(df, output_dir="output"):
    """Save the cleaned DataFrame to Excel"""
    try:
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, "cleaned_car_data.xlsx")
        df.to_excel(output_path, index=False)
        print(f"✅ Saved cleaned data to: {output_path}")
        return True
    except Exception as e:
        print(f"❌ Error saving data: {e}")
        return False

def main():
    # Load and parse data
    input_path = "xlsx_files/bangalore_cars.xlsx"
    df = load_and_parse_data(input_path)
    if df is None:
        return
    
    # Flatten data
    df_flat = flatten_data(df)
    if df_flat is None:
        return
    
    # Clean and transform
    df_clean = clean_and_transform(df_flat)
    if df_clean is None:
        return
    
    # Save cleaned data
    if not save_cleaned_data(df_clean):
        return
    
    # Show sample of processed data
    print("\nSample of processed data:")
    print(df_clean.head())

if __name__ == "__main__":
    main()

✅ Processed 'km' column - removed commas and converted to integer
✅ Processed 'modelYear' - extracted year and month
⚠️ Warning cleaning numeric series: 'DataFrame' object has no attribute 'str'
✅ Processed 'Seats' - converted to Int64
✅ Processed 'Mileage' - converted to <class 'float'>
✅ Processed 'Turning Radius' - converted to <class 'float'>
✅ Processed 'Acceleration' - converted to <class 'float'>
✅ Processed 'Kms Driven' - converted to Int64
✅ Processed 'Engine Displacement' - converted to Int64
✅ Processed 'Torque' - converted to Int64
✅ Processed 'Rear Tread' - converted to Int64
✅ Processed 'Kerb Weight' - converted to Int64
✅ Processed 'Top Speed' - converted to Int64
✅ Processed 'Ground Clearance Unladen' - converted to Int64
✅ Processed 'Displacement' - converted to Int64
✅ Processed 'Seating Capacity' - converted to Int64
✅ Processed 'No Door Numbers' - converted to Int64
✅ Processed 'Wheel Size' - converted to Int64
✅ Processed 'Alloy Wheel Size' - converted to Int64
✅ P

✅ Selected Features and Justifications


| Column               | Description                                   | Justification                                                                 |
|----------------------|-----------------------------------------------|-------------------------------------------------------------------------------|
| `price`              | Selling price of the used car (Target variable) | This is the variable we're predicting.                                        |
| `Kms Driven`         | Total kilometers driven                        | Indicates vehicle usage; higher values usually reduce resale value.           |
| `Mileage`            | Fuel efficiency (e.g., km/l)                   | Higher mileage is appealing and adds value to the car.                        |
| `Fuel Type`          | Petrol, Diesel, CNG, Electric, etc.            | Different fuels affect running costs and buyer demand.                        |
| `Ownership`          | First-hand, second-hand, etc.                  | Fewer owners often indicate better care and increase resale value.            |
| `Insurance Validity` | Remaining insurance period                     | A valid insurance policy adds value and trust for the buyer.                  |
| `Gear Box`           | Number of gears                                | Indicates car performance and class; more gears can mean better performance.  |
| `modelYear`          | Year of manufacture                            | Reflects the age of the vehicle; newer cars tend to sell for higher prices.   |
| `Transmission`       | Manual or Automatic                            | Automatics generally command a higher resale price, especially in cities.     |
| `Seats`              | Number of seats                                | More seating capacity appeals to families and commercial buyers.              |
| `City`               | City where the car is listed                   | Price trends vary by location due to demand, taxes, and road conditions.      |
| `Engine Displacement`| Engine size in CC                              | Affects performance, tax class, and buyer interest.                           |
| `Color`              | Exterior color of the car                      | Certain colors have higher resale appeal depending on regional trends.        |


In [ ]:
import pandas as pd

# Load the Excel file
file_path = "cleaned_car_data.xlsx"  # Use full path if needed
excel_file = pd.ExcelFile(file_path)

# Load the first sheet
df = excel_file.parse(excel_file.sheet_names[0])

# Define wanted columns
wanted_columns = [
    'price',
    'Kms Driven',
    'Mileage',
    'Fuel Type',
    'Ownership',
    'Insurance Validity',
    'Gear Box',
    'modelYear',
    'Transmission',
    'Seats',
    'City',
    'Engine Displacement'
]

# Filter only the columns that exist in the DataFrame
wanted_columns_present = [col for col in wanted_columns if col in df.columns]

# Select and save the cleaned data
df_cleaned = df[wanted_columns_present]
df_cleaned.to_excel("banglore_cars_cleaned.xlsx", sheet_name="banglore_cars", index=False)


**1. Data Import and Structuring**

**a) Import and Concatenate City Datasets**

In [7]:
import pandas as pd
import os
import json
from glob import glob
import ast  # For literal evaluation as fallback

def safe_json_loads(x):
    """Safely parse JSON that might use single quotes or other issues"""
    if not isinstance(x, str) or x.strip() == '':
        return {}
    
    try:
        # First try standard JSON parsing
        return json.loads(x)
    except json.JSONDecodeError:
        try:
            # Try with literal_eval which handles single quotes
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            # If all fails, return empty dict
            print(f"Failed to parse JSON: {x[:100]}...")
            return {}

def process_city_file(csv_file, city_name):
    """Process a single city file and return structured DataFrame"""
    try:
        df = pd.read_csv(csv_file)
        
        # Process JSON columns with more robust parsing
        json_cols = ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']
        for col in json_cols:
            if col in df.columns:
                df[col] = df[col].apply(safe_json_loads)
            else:
                print(f"Column {col} not found in {csv_file}")
                return None
        
        # Flatten nested structures with error handling
        try:
            details = pd.json_normalize(df['new_car_detail'])
            overview = pd.json_normalize(df['new_car_overview'].apply(lambda x: x.get('top', {})))
            features = pd.json_normalize(
                df['new_car_feature'].apply(
                    lambda x: [item.get('value') for item in x.get('top', [])]
                    if isinstance(x.get('top', []), list) 
                    else []
                )
            )
            specs = pd.json_normalize(df['new_car_specs'].apply(lambda x: x.get('top', {})))
            
            # Combine all data
            processed_df = pd.concat([
                df[['car_links']],
                details,
                overview,
                features,
                specs
            ], axis=1)
            
            processed_df['City'] = city_name
            return processed_df
            
        except Exception as e:
            print(f"Error flattening data in {csv_file}: {str(e)}")
            return None
            
    except Exception as e:
        print(f"Error processing {csv_file}: {str(e)}")
        return None

def process_all_cities(data_dir):
    """Process all city files in directory"""
    all_cities = []
    city_files = glob(os.path.join(data_dir, '*.csv'))
    
    if not city_files:
        print(f"No CSV files found in directory: {data_dir}")
        return pd.DataFrame()
    
    for csv_file in city_files:
        city_name = os.path.basename(csv_file).split('_')[0]  # Gets 'bangalore' from 'bangalore_cars.csv'
        print(f"Processing {city_name}...")
        city_df = process_city_file(csv_file, city_name)
        if city_df is not None:
            all_cities.append(city_df)
    
    if not all_cities:
        print("No valid data processed from any files")
        return pd.DataFrame()
    
    try:
        combined_df = pd.concat(all_cities, ignore_index=True)
        print(f"Successfully processed {len(combined_df)} records")
        return combined_df
    except Exception as e:
        print(f"Error concatenating data: {str(e)}")
        return pd.DataFrame()

# Usage:
data_directory = 'csv_files'  # Update this path
combined_data = process_all_cities(data_directory)

if combined_data.empty:
    print("Warning: The resulting DataFrame is empty")
else:
    print("Processing completed successfully")
    print(combined_data.head())

Processing bangalore...
Processing chennai...
Processing delhi...
Processing hyderabad...
Processing jaipur...
Processing kolkata...
Successfully processed 8369 records
Processing completed successfully
                                           car_links  it      ft         bt  \
0  https://www.cardekho.com/used-car-details/used...   0  Petrol  Hatchback   
1  https://www.cardekho.com/buy-used-car-details/...   0  Petrol        SUV   
2  https://www.cardekho.com/used-car-details/used...   0  Petrol  Hatchback   
3  https://www.cardekho.com/buy-used-car-details/...   0  Petrol      Sedan   
4  https://www.cardekho.com/used-car-details/used...   0  Diesel        SUV   

         km transmission  ownerNo      owner      oem               model  \
0  1,20,000       Manual        3  3rd Owner   Maruti      Maruti Celerio   
1    32,706       Manual        2  2nd Owner     Ford       Ford Ecosport   
2    11,949       Manual        1  1st Owner     Tata          Tata Tiago   
3    17,794   

**2. Handling Missing Values**

In [8]:
def handle_missing_values(df):
    """Handle missing values in the dataset"""
    
    # Numerical columns - fill with median
    numerical_cols = ['km', 'modelYear', 'ownerNo', 'priceActual']
    for col in numerical_cols:
        if col in df.columns:
            # Clean numerical columns first (remove commas, non-numeric chars)
            df[col] = pd.to_numeric(df[col].astype(str).str.replace('[^\d.]', '', regex=True), errors='coerce')
            df[col].fillna(df[col].median(), inplace=True)
    
    # Categorical columns - fill with mode or 'Unknown'
    categorical_cols = ['ft', 'bt', 'transmission', 'owner', 'oem', 'model']
    for col in categorical_cols:
        if col in df.columns:
            df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else 'Unknown', inplace=True)
    
    # Price handling - extract numerical value from string
    if 'price' in df.columns:
        df['price'] = df['price'].str.extract(r'₹\s*([\d,.]+)')[0]
        df['price'] = pd.to_numeric(df['price'].str.replace(',', ''), errors='coerce')
        df['price'].fillna(df['price'].median(), inplace=True)
    
    return df

# Apply missing value handling
cleaned_data = handle_missing_values(combined_data)

C:\Users\USER\AppData\Local\Temp\ipykernel_1464\2525035043.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_1464\2525035043.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

**Standardizing Data Formats**

In [ ]:
def standardize_data_formats(df):
    """Standardize data formats across columns"""
    
    # Convert kms driven to numeric
    if 'km' in df.columns:
        df['km'] = df['km'].astype(str).str.replace('Kms', '').str.replace(',', '').str.strip()
        df['km'] = pd.to_numeric(df['km'], errors='coerce')
    
    # Convert engine displacement to numeric (cc)
    if 'engineDisplacement' in df.columns:
        df['engineDisplacement_cc'] = df['engineDisplacement'].str.extract(r'(\d+)').astype(float)
    
    # Extract power values
    if 'maxPower' in df.columns:
        df['maxPower_bhp'] = df['maxPower'].str.extract(r'([\d.]+)').astype(float)
    
    # Convert torque to numeric
    if 'torque' in df.columns:
        df['torque_nm'] = df['torque'].str.extract(r'([\d.]+)').astype(float)
    
    # Convert year columns to datetime
    year_cols = ['modelYear', 'registrationYear']
    for col in year_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='%Y', errors='coerce').dt.year
    
    # Standardize owner information
    if 'owner' in df.columns:
        df['owner'] = df['owner'].str.replace(r'\d(st|nd|rd|th)\s*Owner', lambda m: m.group(1) + ' Owner', regex=True)
    
    return df

# Apply standardization
standardized_data = standardize_data_formats(cleaned_data)

**Feature Engineering**

In [ ]:
def perform_feature_engineering(df):
    """Create new features from existing data"""
    
    # Calculate car age
    current_year = pd.Timestamp.now().year
    if 'modelYear' in df.columns:
        df['car_age'] = current_year - df['modelYear']
    
    # Create binary features from car features
    if 'features' in df.columns:
        all_features = set()
        for feature_list in df['features']:
            if isinstance(feature_list, list):
                all_features.update(feature_list)
        
        for feature in all_features:
            df[f'has_{feature}'] = df['features'].apply(lambda x: feature in x if isinstance(x, list) else False)
    
    # Create transmission type binary columns
    if 'transmission' in df.columns:
        df['is_automatic'] = df['transmission'].str.contains('Automatic', case=False)
        df['is_manual'] = df['transmission'].str.contains('Manual', case=False)
    
    return df

# Apply feature engineering
final_data = perform_feature_engineering(standardized_data)

**5. Saving Processed Data**

In [ ]:
# Save the processed data
final_data.to_csv('processed_car_data.csv', index=False)
# Or save to pickle for preserving data types
final_data.to_pickle('processed_car_data.pkl')

**4. Encoding Categorical Variables**

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

def encode_categorical_variables(df):
    """Encode categorical variables using appropriate techniques"""
    
    # Identify categorical columns
    nominal_cats = ['oem', 'model', 'City', 'fuelType', 'transmissionType']  # No inherent ordering
    ordinal_cats = ['owner']  # Has meaningful order (1st > 2nd > 3rd Owner)
    
    # One-hot encoding for nominal variables
    onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    onehot_encoded = onehot_encoder.fit_transform(df[nominal_cats])
    
    # Create DataFrame from one-hot encoded data
    onehot_df = pd.DataFrame(
        onehot_encoded,
        columns=onehot_encoder.get_feature_names_out(nominal_cats)
    )
    
    # Ordinal encoding for ordered categories
    owner_order = ['4th Owner', '3rd Owner', '2nd Owner', '1st Owner']  # Lower is better
    ordinal_encoder = OrdinalEncoder(categories=[owner_order])
    df['owner_encoded'] = ordinal_encoder.fit_transform(df[['owner']])
    
    # Label encoding for binary categories (if any)
    binary_cols = ['is_automatic', 'is_manual']  # Example binary columns
    for col in binary_cols:
        if col in df.columns:
            df[col] = LabelEncoder().fit_transform(df[col])
    
    # Combine encoded data with original numerical data
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    final_df = pd.concat([
        df[numerical_cols],
        onehot_df,
        df[['owner_encoded']]
    ], axis=1)
    
    return final_df, onehot_encoder, ordinal_encoder

# Apply encoding
encoded_data, ohe, ord_enc = encode_categorical_variables(final_data)

**5. Normalizing Numerical Features**

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def normalize_numerical_features(df, scaling_method='minmax'):
    """Normalize numerical features to common scale"""
    
    # Select numerical columns (excluding target and encoded columns)
    numerical_cols = [col for col in df.columns 
                    if df[col].dtype in ['int64', 'float64'] 
                    and col != 'price'  # Exclude target variable
                    and not col.endswith('_encoded')]
    
    # Apply selected scaling method
    if scaling_method == 'minmax':
        scaler = MinMaxScaler()
    else:  # standard
        scaler = StandardScaler()
    
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    
    return df, scaler

# Apply normalization (choose either 'minmax' or 'standard')
normalized_data, feature_scaler = normalize_numerical_features(encoded_data, 'minmax')

**6. Removing Outliers**

In [ ]:
from scipy import stats

def handle_outliers(df, method='iqr', threshold=1.5):
    """Identify and handle outliers in numerical features"""
    
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    
    if method == 'iqr':
        # IQR method
        for col in numerical_cols:
            if col != 'price':  # Don't remove based on target variable
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - threshold * IQR
                upper_bound = Q3 + threshold * IQR
                
                # Cap outliers instead of removing
                df[col] = df[col].clip(lower_bound, upper_bound)
    
    elif method == 'zscore':
        # Z-score method
        for col in numerical_cols:
            if col != 'price':
                z_scores = stats.zscore(df[col])
                df = df[(np.abs(z_scores) < threshold)]
    
    return df

# Apply outlier handling (choose either 'iqr' or 'zscore')
cleaned_data = handle_outliers(normalized_data, 'iqr', 1.5)

**Complete Data Processing Pipeline**

In [ ]:
def full_data_processing_pipeline(data_dir, city_files):
    """Complete data processing pipeline"""
    
    # 1. Import and concatenate city datasets
    combined_data = process_all_cities(data_dir, city_files)
    
    # 2. Handle missing values
    cleaned_data = handle_missing_values(combined_data)
    
    # 3. Standardize data formats
    standardized_data = standardize_data_formats(cleaned_data)
    
    # 4. Feature engineering
    final_data = perform_feature_engineering(standardized_data)
    
    # 5. Encode categorical variables
    encoded_data, ohe, ord_enc = encode_categorical_variables(final_data)
    
    # 6. Normalize numerical features
    normalized_data, feature_scaler = normalize_numerical_features(encoded_data)
    
    # 7. Handle outliers
    processed_data = handle_outliers(normalized_data)
    
    return processed_data, ohe, ord_enc, feature_scaler

# Example usage:
data_dir = 'path/to/city/files'
city_files = ['Chennai.csv', 'Mumbai.csv', 'Delhi.csv']  # etc.
final_processed_data, ohe, ord_enc, scaler = full_data_processing_pipeline(data_dir, city_files)